In [1]:
import numpy as np
import matplotlib.pyplot as plt
import datajoint as dj
import pandas as pd
import re
import json
import urllib

In [2]:
starting_link = 'https://neuromancer-seung-import.appspot.com/#!%7B%22layers%22:%5B%7B%22source%22:%22graphene://https://minnie.microns-daf.com/proxy/minniev1_em%22%2C%22type%22:%22image%22%2C%22blend%22:%22default%22%2C%22shaderControls%22:%7B%7D%2C%22name%22:%22minniev1_em%22%7D%2C%7B%22source%22:%22precomputed://gs://microns-seunglab/minnie65/seg_minnie65_0%22%2C%22type%22:%22segmentation%22%2C%22colorSeed%22:2940450712%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22seg%22%7D%2C%7B%22tool%22:%22annotateLine%22%2C%22selectedAnnotation%22:%7B%22id%22:%22f5faca4325acb092b58b7dc0537540731753c48b%22%7D%2C%22annotationColor%22:%22#ffffff%22%2C%22type%22:%22annotation%22%2C%22annotations%22:%5B%7B%22pointA%22:%5B282609.28125%2C339508%2C14884%5D%2C%22pointB%22:%5B229619.796875%2C339508%2C27882%5D%2C%22type%22:%22line%22%2C%22id%22:%221fc63d14acf21afc7be1bf019ad228732b8d0f26%22%2C%22tagIds%22:%5B%5D%7D%2C%7B%22pointA%22:%5B297635.59375%2C339508%2C14884%5D%2C%22pointB%22:%5B392616.71875%2C339508%2C27882%5D%2C%22type%22:%22line%22%2C%22id%22:%22918338b03490963acb90a5245f02a353a41fae84%22%2C%22tagIds%22:%5B%5D%7D%2C%7B%22pointA%22:%5B282609.28125%2C339508%2C14884%5D%2C%22pointB%22:%5B282600.15625%2C87589.9453125%2C14884%5D%2C%22type%22:%22line%22%2C%22id%22:%22fea8955e5bd0d9376a14a0a63f44ef502c68f581%22%2C%22tagIds%22:%5B%5D%7D%2C%7B%22pointA%22:%5B297635.59375%2C339508%2C14884%5D%2C%22pointB%22:%5B298099.09375%2C87556%2C14884%5D%2C%22type%22:%22line%22%2C%22id%22:%22363e552e5d3290650a7b877e1cb28c82092f1b60%22%2C%22tagIds%22:%5B%5D%7D%2C%7B%22pointA%22:%5B229619.796875%2C339508%2C27882%5D%2C%22pointB%22:%5B228688.6875%2C87556%2C27882%5D%2C%22type%22:%22line%22%2C%22id%22:%228c7a1f26dabce2b261c36128035b647d8c1fcb36%22%2C%22tagIds%22:%5B%5D%7D%2C%7B%22pointA%22:%5B392616.71875%2C339508%2C27882%5D%2C%22pointB%22:%5B392170.96875%2C87556%2C27882%5D%2C%22type%22:%22line%22%2C%22id%22:%223aae3309f646e6362959b6ea778639484e836aef%22%2C%22tagIds%22:%5B%5D%7D%2C%7B%22pointA%22:%5B228688.6875%2C87556%2C27882%5D%2C%22pointB%22:%5B282600.15625%2C87589.9453125%2C14884%5D%2C%22type%22:%22line%22%2C%22id%22:%22e287f6fe233d27cdb37662681159cc7dfb4d91c3%22%2C%22tagIds%22:%5B%5D%7D%2C%7B%22pointA%22:%5B298099.09375%2C87556%2C14884%5D%2C%22pointB%22:%5B392170.96875%2C87556%2C27882%5D%2C%22type%22:%22line%22%2C%22id%22:%22f5faca4325acb092b58b7dc0537540731753c48b%22%2C%22tagIds%22:%5B%5D%7D%5D%2C%22annotationTags%22:%5B%5D%2C%22voxelSize%22:%5B4%2C4%2C40%5D%2C%22name%22:%22area-boundaries%22%7D%5D%2C%22navigation%22:%7B%22pose%22:%7B%22position%22:%7B%22voxelSize%22:%5B4%2C4%2C40%5D%2C%22voxelCoordinates%22:%5B258295.15625%2C209637.078125%2C22925%5D%7D%2C%22orientation%22:%5B0%2C-0.7071067690849304%2C0%2C0.7071067690849304%5D%7D%2C%22zoomFactor%22:3616.179299933718%7D%2C%22perspectiveZoom%22:43225.153342864134%2C%22showSlices%22:false%2C%22gpuMemoryLimit%22:2500000000%2C%22systemMemoryLimit%22:2500000000%2C%22jsonStateServer%22:%22https://www.dynamicannotationframework.com/nglstate/post%22%2C%22layout%22:%223d%22%7D'

In [3]:
def html_to_json(url_string, return_parsed_url=False, fragment_prefix='!'):
    # Parse neuromancer url to logically separate the json state dict from the rest of it.
    full_url_parsed = urllib.parse.urlparse(url_string)
    # Decode percent-encoding in url, and skip "!" from beginning of string.
    decoded_fragment = urllib.parse.unquote(full_url_parsed.fragment)
    if decoded_fragment.startswith(fragment_prefix):
        decoded_fragment = decoded_fragment[1:]
    # Load the json state dict string into a python dictionary.
    json_state_dict = json.loads(decoded_fragment)

    if return_parsed_url:
        return json_state_dict, full_url_parsed
    else:
        return json_state_dict

def json_to_url(json_dict, fragment_prefix='https://neuromancer-seung-import.appspot.com/#!'):
    loaded_state = json.loads(json_dict)
    return f"{fragment_prefix}{json.dumps(loaded_state)}"

def add_point_annotations(provided_link, ano_name, ano_list, voxelsize=[4,4,40], descriptions=None, color='#f1ff00', overwrite=True):
    # format annotation list
    ano_list_dict = []
    if ano_list.ndim<2:
        ano_list = np.expand_dims(ano_list,0)
    if ano_list.ndim>2:
        return print('The annotation list must be 1D or 2D')
    
    if descriptions is not None:
        for i, (centroid, desc) in enumerate(zip(ano_list.tolist(), descriptions)):
            dict_to_add = {'point':centroid, 'type':'point', 'id':str(i+1), 'description':f'{desc}', 'tagIds':[]}
            ano_list_dict.append(dict_to_add)
            print(dict_to_add)
            
    for i, centroid in enumerate(ano_list.tolist()):
            ano_list_dict.append({'point':centroid, 'type':'point', 'id':str(i+1), 'tagIds':[]})

    json_data, parsed_url = html_to_json(provided_link, return_parsed_url=True)
    # if annotation layer doesn't exist, create it
    if re.search(ano_name,json.dumps(json_data)) is None:
        json_data['layers'].append({'tool': 'annotatePoint',
                               'type': 'annotation',
                               'annotations': [],
                               'annotationColor': color,
                               'annotationTags': [],
                               'voxelSize': voxelsize,
                               'name': ano_name})
        # print('annotation layer does not exist... creating it')
    annotation_dict = list(filter(lambda d: d['name'] == ano_name, json_data['layers']))
    annotation_ind = np.where(np.array(json_data['layers']) == annotation_dict)[0][0].squeeze()
    # test if voxel size of annotation matches provided voxel size
    if json_data['layers'][annotation_ind]['voxelSize']!=voxelsize:
        return print('The annotation layer already exists but does not match your provided voxelsize')
    # add annotations
    if overwrite:
        json_data['layers'][annotation_ind]['annotations'] = ano_list_dict
    else:
        json_data['layers'][annotation_ind]['annotations'].extend(ano_list_dict)

    return urllib.parse.urlunparse([parsed_url.scheme, parsed_url.netloc, parsed_url.path, parsed_url.params, parsed_url.query, '!'+ urllib.parse.quote(json.dumps(json_data))])

def add_segments(provided_link, segments, seg_name='seg', overwrite=True, color=None):
    json_data, parsed_url = html_to_json(provided_link, return_parsed_url=True)
    seg_strings = []
    for seg in segments:
        seg_strings.append(seg.astype(np.str))
    segmentation_layer = list(filter(lambda d: np.logical_and(d['type'] == 'segmentation', d['name'] == seg_name), json_data['layers']))
    if len(segmentation_layer) == 0:
        # print('segmentation layer does not exist... creating it')
        json_data['layers'].append({
        "source": "precomputed://gs://microns-seunglab/minnie65/seg_minnie65_0",
        "type": "segmentation",
        "colorSeed": 2940450712,
        "segments": [],
        "skeletonRendering": {
        "mode2d": "lines_and_points",
        "mode3d": "lines"
          },
       "name": f"{seg_name}"
        })
        segmentation_layer = list(filter(lambda d: np.logical_and(d['type'] == 'segmentation', d['name'] == seg_name), json_data['layers']))
    if re.search('segments',json.dumps(json_data)) is None:
        segmentation_layer[0].update({'segments':[]})
    if overwrite:
        segmentation_layer[0]['segments'] = seg_strings
    else:
        segmentation_layer[0]['segments'].extend(seg_strings)
    if color is not None:
        if re.search('segmentColors',json.dumps(json_data)) is None:
            segmentation_layer[0].update({'segmentColors':{}})
        color_dict = {}
        for seg in segments:
            color_dict.update({str(seg):color})
        segmentation_layer[0]['segmentColors'] = color_dict
            
    return urllib.parse.urlunparse([parsed_url.scheme, parsed_url.netloc, parsed_url.path, parsed_url.params, parsed_url.query, '!'+ urllib.parse.quote(json.dumps(json_data))])

In [4]:
# with_seg = add_segments(starting_link, np.array([98816821846776280, 99588816515836931])) # enter list of segment id's here
# with_syn = add_point_annotations(with_seg, 'synapse', np.array([262685, 221048, 22164])) # enter synapse coordinates here
# print(with_syn)

In [34]:
# load in the data
loaded_direct_data = np.load("direct_connection_data.npz",allow_pickle=True)
direct_connections_postsyn = loaded_direct_data["direct_connections_postsyn"]
direct_connections_presyn = loaded_direct_data["direct_connections_presyn"]
direct_connections_coordinates = loaded_direct_data["direct_connections_coordinates"]

In [37]:
total_soma_to_soma_distance = []
for curr_presyn,curr_postsyn in zip(direct_connections_presyn,direct_connections_postsyn):
    curr_distance = np.linalg.norm(soma_center_dict[curr_presyn]-soma_center_dict[curr_postsyn])
    total_soma_to_soma_distance.append(curr_distance)



NameError: name 'soma_center_dict' is not defined

In [ ]:
from matplotlib import pyplot as plt
fig,ax = plt.subplots(1,1)
bins = plt.hist(np.array(total_soma_to_soma_distance)/1000,bins=100)
ax.set_xlabel("Distance from presyn to postsyn soma (um)")
ax.set_ylabel("frequency")
ax.set_title("Distance from presyn to postsyn soma frequency chart")
plt.show()

In [ ]:
current_counter = 500000
with_seg = add_segments(starting_link, np.array([direct_connections_presyn[current_counter]
                                                ,direct_connections_postsyn[current_counter]])) # enter list of segment id's here
with_syn = add_point_annotations(with_seg, 'synapse', direct_connections_coordinates) # enter synapse coordinates here

# Just visualizing the direct connection synapses

In [19]:
subsampling_the_synapses = direct_connections_coordinates[0::1000]
len(subsampling_the_synapses)

1130

In [20]:
current_counter = 500000
with_seg = add_segments(starting_link, np.array([direct_connections_presyn[current_counter]
                                                ,direct_connections_postsyn[current_counter]])) # enter list of segment id's he
with_syn = add_point_annotations(with_seg, 'synapse', subsampling_the_synapses) # enter synapse coordinates here

In [21]:
print(with_syn)

https://neuromancer-seung-import.appspot.com/#!%7B%22layers%22%3A%20%5B%7B%22source%22%3A%20%22graphene%3A//https%3A//minnie.microns-daf.com/proxy/minniev1_em%22%2C%20%22type%22%3A%20%22image%22%2C%20%22blend%22%3A%20%22default%22%2C%20%22shaderControls%22%3A%20%7B%7D%2C%20%22name%22%3A%20%22minniev1_em%22%7D%2C%20%7B%22source%22%3A%20%22precomputed%3A//gs%3A//microns-seunglab/minnie65/seg_minnie65_0%22%2C%20%22type%22%3A%20%22segmentation%22%2C%20%22colorSeed%22%3A%202940450712%2C%20%22skeletonRendering%22%3A%20%7B%22mode2d%22%3A%20%22lines_and_points%22%2C%20%22mode3d%22%3A%20%22lines%22%7D%2C%20%22name%22%3A%20%22seg%22%2C%20%22segments%22%3A%20%5B%2294101841578627886%22%2C%20%2297756618028163542%22%5D%7D%2C%20%7B%22tool%22%3A%20%22annotateLine%22%2C%20%22selectedAnnotation%22%3A%20%7B%22id%22%3A%20%22f5faca4325acb092b58b7dc0537540731753c48b%22%7D%2C%20%22annotationColor%22%3A%20%22%23ffffff%22%2C%20%22type%22%3A%20%22annotation%22%2C%20%22annotations%22%3A%20%5B%7B%22pointA%22%3A%2

# Visualizing the soma centers of the 1 soma

In [ ]:
# Just visualizing the single soma cells


In [33]:
import numpy as np
single_soma_data = np.load("single_soma_centers.npz")
single_soma_coordinates = single_soma_data["single_soma_coordinates"]
single_soma_coordinates.shape

ValueError: Object arrays cannot be loaded when allow_pickle=False

In [31]:
single_soma_coordinates_subsampled = single_soma_coordinates[0::40]
len(single_soma_coordinates_subsampled)

1188

In [32]:
current_counter = 500000
with_seg = add_segments(starting_link, np.array([direct_connections_presyn[current_counter]
                                                ,direct_connections_postsyn[current_counter]])) # enter list of segment id's he
with_syn = add_point_annotations(with_seg, 'synapse', single_soma_coordinates_subsampled) # enter synapse coordinates here
print(with_syn)

https://neuromancer-seung-import.appspot.com/#!%7B%22layers%22%3A%20%5B%7B%22source%22%3A%20%22graphene%3A//https%3A//minnie.microns-daf.com/proxy/minniev1_em%22%2C%20%22type%22%3A%20%22image%22%2C%20%22blend%22%3A%20%22default%22%2C%20%22shaderControls%22%3A%20%7B%7D%2C%20%22name%22%3A%20%22minniev1_em%22%7D%2C%20%7B%22source%22%3A%20%22precomputed%3A//gs%3A//microns-seunglab/minnie65/seg_minnie65_0%22%2C%20%22type%22%3A%20%22segmentation%22%2C%20%22colorSeed%22%3A%202940450712%2C%20%22skeletonRendering%22%3A%20%7B%22mode2d%22%3A%20%22lines_and_points%22%2C%20%22mode3d%22%3A%20%22lines%22%7D%2C%20%22name%22%3A%20%22seg%22%2C%20%22segments%22%3A%20%5B%2294101841578627886%22%2C%20%2297756618028163542%22%5D%7D%2C%20%7B%22tool%22%3A%20%22annotateLine%22%2C%20%22selectedAnnotation%22%3A%20%7B%22id%22%3A%20%22f5faca4325acb092b58b7dc0537540731753c48b%22%7D%2C%20%22annotationColor%22%3A%20%22%23ffffff%22%2C%20%22type%22%3A%20%22annotation%22%2C%20%22annotations%22%3A%20%5B%7B%22pointA%22%3A%2